In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

Your Name and PRN:
- Name: ______________________
- PRN : ______________________
- Date: ______________________

# Image Processing with Neural Network

## Assignment: A04
### Take Neural Network with :
- Multiple hidden layers 
- Activation function of your choice


### Assignment
- **Q1**: Is this model more accurate compared to previous model?
- **Q2**: Prepare table by changing number of neurons in hidden layer, learning rate and observe change in results. Also comment on your results.

|#|Dimension of hidden layer|Learning rate|Training Accuracy|Test Accuracy|Comment|
|:-:|:-:|:-:|:-:|:-:|:--|
|1|5-5-4-3-2|0.1|0.97|0.96|Base case||1|4|0.1|0.97|0.96|Base case|
|2|10-5-4-3-2|1|???|???|???|
|...|...|...|...|...|...|
|n|...|...|...|...|...|

In [ ]:
###-----------------
### Import libraries
###-----------------
from pathlib import Path  # Import Path for file system path operations and management
import numpy as np  # Import NumPy for numerical computations and array operations
import pandas as pd  # Import Pandas for data manipulation and analysis with DataFrames
import matplotlib.pyplot as plt  # Import Matplotlib for creating static, interactive visualizations
import seaborn as sns  # Import Seaborn for statistical data visualization built on Matplotlib
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split  # Import function to split dataset into training and testing subsets
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             f1_score)  # Import function to calculate various metric
from sklearn.preprocessing import StandardScaler

In [ ]:
# --- Global Parameters ---
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
rng = np.random.default_rng(seed=RANDOM_STATE)
TEST_SIZE  = 0.2
NOISE=0.2
EPOCHS=10000 # Reduced epochs for faster execution, results in table are based on this.
N_SAMPLE=1000
ALPHA=0.01 # Default Alpha

CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
def fn_plot_decision_boundary(X: np.ndarray, y: np.ndarray, model:dict, predict):
    """
    Plots the decision boundary for a classification model along with the data points.

    Args:
        X (np.ndarray): Input feature matrix with shape (n_samples, 3)
        wts (np.ndarray): Weights matrix
        pred_function: Function to predict using weights and datapoints
    """

    # Initialize the figure with specified dimensions
    fig, ax = plt.subplots( figsize=(8, 5) )

    # Small increment value to create a fine grid for smooth decision boundary
    dm = 0.05
    padding = 2 * dm

    # Calculate the range for x-axis (first feature) with padding
    x_min, x_max = X[:, 0].min() - padding, X[:, 0].max() + padding

    # Calculate the range for y-axis (second feature) with padding
    y_min, y_max = X[:, 1].min() - padding, X[:, 1].max() + padding

    # Create a mesh grid covering the entire feature space
    xx, yy = np.meshgrid(np.arange(x_min, x_max, dm),
                         np.arange(y_min, y_max, dm))

    # Flatten the mesh grid arrays and stack them column-wise to create coordinate pairs
    XX = np.c_[xx.ravel(), yy.ravel()] # Resulting shape: (n_points, 2)

    # # Add a column of ones to the coordinate array for the bias term
    # XX = np.hstack((XX, np.ones((XX.shape[0], 1)))) # make array compatible

    # List to collect predictions for each point in the mesh grid
    y_pred = predict(model, XX)

    # Iterate over each coordinate point in the mesh grid
    # for row in XX:
    #     # Make prediction for the current coordinate using global 'weights' and 'predict' function
    #     y_p = predict(model, X)
    #     y_pred.append(y_p)


    # Reshape predictions to match the original mesh grid dimensions
    Z = np.array(y_pred).reshape(xx.shape)

    # Create filled contour plot showing the decision regions
    ax.contourf(xx, yy, Z, alpha=0.6, cmap='rainbow')

    # Scatter plot of the actual data points, colored by their true class labels
    ax.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k', cmap='rainbow')

    # Set plot title and axis labels
    ax.set_title('Decision Boundary')
    ax.set_xlabel('A')
    ax.set_ylabel('B')

    # Display the final plot
    plt.show()

In [ ]:
X,y=make_moons(n_samples=N_SAMPLE,noise=NOISE,random_state=RANDOM_STATE,shuffle=True)
type(X),type(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
def fn_softmax(z):
    exp_score = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_score / np.sum(exp_score, axis=1, keepdims=True)

def fn_activ(z):
    return np.tanh(z)

def fn_activ_prime(z):
    return 1.0 - np.tanh(z)**2

def calculate_a(X_p, model):
    W1, W2, W3, W4, W5 = model['W1'], model['W2'], model['W3'], model['W4'], model['W5']
    b1, b2, b3, b4, b5 = model['b1'], model['b2'], model['b3'], model['b4'], model['b5']

    # Forward Propagation
    z1 = X_p.dot(W1) + b1
    a1 = fn_activ(z1)

    z2 = a1.dot(W2) + b2
    a2 = fn_activ(z2)

    z3 = a2.dot(W3) + b3
    a3 = fn_activ(z3)

    z4 = a3.dot(W4) + b4
    a4 = fn_activ(z4)

    z5 = a4.dot(W5) + b5
    a5 = fn_softmax(z5)
    return a5, a4, a3, a2, a1, z5, z4, z3, z2, z1

def predict(model, X_p):
    a5, _, _, _, _, _, _, _, _, _ = calculate_a(X_p, model)
    return a5.argmax(axis=1)

def build_model_and_train(h_dim_config, X_tr, y_tr, X_ts, y_ts, alpha, n_epochs, random_state=42):
    # Setup for 4 hidden layers
    rng = np.random.default_rng(seed=random_state)
    h_dim = [X_tr.shape[1]] + h_dim_config + [y_tr.shape[1]]

    W1 = rng.random((h_dim[0], h_dim[1])) / np.sqrt(h_dim[0])
    W2 = rng.random((h_dim[1], h_dim[2])) / np.sqrt(h_dim[1])
    W3 = rng.random((h_dim[2], h_dim[3])) / np.sqrt(h_dim[2])
    W4 = rng.random((h_dim[3], h_dim[4])) / np.sqrt(h_dim[3])
    W5 = rng.random((h_dim[4], h_dim[5])) / np.sqrt(h_dim[4])

    b1, b2, b3, b4, b5 = [np.zeros((1, d)) for d in h_dim[1:]]
    m = X_tr.shape[0]

    for i in range(n_epochs):
        model = {'W1': W1, 'W2': W2, 'W3': W3, 'W4': W4, 'W5': W5,
                 'b1': b1, 'b2': b2, 'b3': b3, 'b4': b4, 'b5': b5}

        a5, a4, a3, a2, a1, z5, z4, z3, z2, z1 = calculate_a(X_tr, model)

        # Back Propagation
        dz5, dw5, db5, da4 = [a5 - y_tr, (a4.T).dot(dz5), np.sum(dz5, axis=0, keepdims=True), dz5.dot(W5.T)]
        dz4, dw4, db4, da3 = [da4 * fn_activ_prime(z4), (a3.T).dot(dz4), np.sum(dz4, axis=0, keepdims=True), dz4.dot(W4.T)]
        dz3, dw3, db3, da2 = [da3 * fn_activ_prime(z3), (a2.T).dot(dz3), np.sum(dz3, axis=0, keepdims=True), dz3.dot(W3.T)]
        dz2, dw2, db2, da1 = [da2 * fn_activ_prime(z2), (a1.T).dot(dz2), np.sum(dz2, axis=0, keepdims=True), dz2.dot(W2.T)]
        dz1, dw1, db1 = [da1 * fn_activ_prime(z1), (X_tr.T).dot(dz1), np.sum(dz1, axis=0, keepdims=True)]

        # Parameter Update
        W1 -= alpha * dw1 / m; b1 -= alpha * db1 / m
        W2 -= alpha * dw2 / m; b2 -= alpha * db2 / m
        W3 -= alpha * dw3 / m; b3 -= alpha * db3 / m
        W4 -= alpha * dw4 / m; b4 -= alpha * db4 / m
        W5 -= alpha * dw5 / m; b5 -= alpha * db5 / m

    final_model = {'W1': W1, 'W2': W2, 'W3': W3, 'W4': W4, 'W5': W5,
                   'b1': b1, 'b2': b2, 'b3': b3, 'b4': b4, 'b5': b5}

    # Evaluate
    y_tr_pred = predict(final_model, X_tr)
    y_ts_pred = predict(final_model, X_ts)

    tr_acc = accuracy_score(y_tr.argmax(axis=1), y_tr_pred)
    ts_acc = accuracy_score(y_ts.argmax(axis=1), y_ts_pred)

    return tr_acc, ts_acc

In [ ]:
experiments = [
    {"id": 1, "h_dim_config": [5, 5, 4, 3], "alpha": 0.1},
    {"id": 2, "h_dim_config": [10, 5, 4, 3], "alpha": 1.0},
    {"id": 3, "h_dim_config": [50, 20, 10, 5], "alpha": 0.05},
    {"id": 4, "h_dim_config": [100, 100, 100, 100], "alpha": 0.001}
]

results = []
# Run each experiment and append to results list
for exp in experiments:
    tr_acc, ts_acc = build_model_and_train(
        h_dim_config=exp['h_dim_config'],
        X_tr=X_train,
        y_tr=y_train,
        X_ts=X_test,
        y_ts=y_test,
        alpha=exp['alpha'],
        n_epochs=EPOCHS
    )